In [1]:
import torch
import torch_geometric
from torch_geometric.nn import GAE
from RNARepLearn.datasets import CombinedRfamDataset
from RNARepLearn.modules import RPINetEncoder
from RNARepLearn.utils import train_val_test_loaders

/home/icb/vnicolas.goedert/miniconda3/envs/RNARepLearn/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ds = CombinedRfamDataset("/lustre/groups/crna01/workspace/nicolas_msc/RNARepLearn/rfam/data/raw/processed/release-14.8",None, "All")
train, val, test = train_val_test_loaders(ds, 0.6, 0.001, 0.399, 32)

Processing complete
Training:	439537
Test:	292292
Validation:	734


Processing...
Done!


In [3]:
encoder = RPINetEncoder(4,32,3,6)
model = GAE(encoder).double()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
optimizer = torch.optim.Adam(model.parameters(), lr=0.002, weight_decay=5e-4)
model.to(device)

GAE(
  (encoder): RPINetEncoder(
    (body): Sequential(
      (0): RPINetGNNLayer()
      (1): RPINetGNNLayer()
      (2): RPINetGNNLayer()
      (3): LSTM(32, 32, bidirectional=True)
    )
  )
  (decoder): InnerProductDecoder()
)

In [5]:
for epoch in range(10):
    for idx, batch in enumerate(train):
        optimizer.zero_grad()
        batch.to(device)
        z = model.encode(batch, batch.edge_index)
        
        loss = model.recon_loss(z, batch.edge_index)
        
        loss.backward()
        optimizer.step()
        
        print("epoch "+str(epoch)+"  batch"+ idx+"  "+str(float(loss)))
        

TypeError: RPINetEncoder.forward() takes 2 positional arguments but 3 were given